In [1]:
import numpy as np


#加载数据
def load_data():
    #这个数据集,只有数字0,1,2,3,一共800行784列
    with open('mnist_cut.csv') as fr:
        lines = fr.readlines()

    x = np.empty((len(lines), 784), dtype=float)
    y = np.empty(len(lines), dtype=int)

    for i in range(len(lines)):
        line = lines[i].strip().split(',')
        x[i] = line[1:]
        y[i] = line[0]

    #归一化
    x /= 255

    return x, y


x, y = load_data()
x[:5], y[:5], x.shape, y.shape

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([2, 1, 0, 1, 0]),
 (800, 784),
 (800,))

In [2]:
#查看各类的数量
for i in range(4):
    print(i, (y == [i]).sum())

0 168
1 225
2 209
3 198


In [3]:
N, M = x.shape

#这里训练的是ovo的多分类,一共6个二分类器,就是4个类两两排列组合
params = ['01', '02', '03', '12', '13', '23']
ws = {}
bs = {}
for i in params:
    ws[i] = np.empty(M)
    ws[i].fill(1 / M)
    bs[i] = 0.0

In [4]:
#预测函数,显然,他一次只能做一次二分类判断
def predict(params, x):
    z = ws[params].dot(x) + bs[params]
    return 1 / (1 + np.exp(-z))


predict('01', x[0])

0.5360144532148693

In [5]:
#这个log函数是为了避免log0.
def log(p):
    if p == 0:
        p = 1e-20
    return np.log(p)

#loss函数
def get_loss(params):
    loss = 0
    for i in range(N):
        #只有是自己二分类中的一类才计算loss,其他类的loss计算没有意义
        if str(y[i]) in params:
            p = predict(params, x[i])
            #二分类中的第一类是0,另一类是1
            d = 0 if str(y[i]) == params[0] else 1
            loss += d * log(p) + (1 - d) * log(1 - p)

    return loss


get_loss('01')

-278.37911686298537

In [6]:
#暴力求梯度法
#同样的,因为这个程序中实际是6个二分类器,所以这里的梯度,也要指定是哪一个的
def get_gradient(params):
    global w
    global b
    upsilon = 1e-2

    gradient_w = np.empty(M)

    for i in range(M):
        l1 = get_loss(params)
        ws[params][i] += upsilon
        l2 = get_loss(params)
        ws[params][i] -= upsilon
        gradient_w[i] = (l2 - l1) / upsilon

    l1 = get_loss(params)
    bs[params] += upsilon
    l2 = get_loss(params)
    bs[params] -= upsilon
    gradient_b = (l2 - l1) / upsilon

    return gradient_w, gradient_b


get_gradient('01')[0][:100]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.42421197,  0.68744269,  0.37

In [7]:
#训练,因为是6个分类器,显然要训练6次
for param in params:
    for i in range(50):
        g_w, g_b = get_gradient(param)
        ws[param] += g_w * 1e-2
        bs[param] += g_b * 1e-1

        if i % 10 == 0:
            print(param, i, get_loss(param))

01 0 -0.09721305308799165
01 10 -0.07259197823765838
01 20 -0.0584861844270012
01 30 -0.04940968066272462
01 40 -0.04311613792244203
02 0 -57.20651278101825
02 10 -93.89839611020031
02 20 -28.356635022821543
02 30 -6.679625478918654
02 40 -1.5635731356544724
03 0 -19.554040865591535
03 10 -2.7450048366824467
03 20 -1.2179360222932507
03 30 -0.8372873206165021
03 40 -0.6699565754595802
12 0 -55.64806841835235
12 10 -5.870600759079906
12 20 -2.6972055519559426
12 30 -1.868802480537476
12 40 -1.4838556615987815
13 0 -26.619788831876587
13 10 -4.020637036792894
13 20 -2.47888129190784
13 30 -1.8803402761977672
13 40 -1.5489130504073492
23 0 -402.3800096483134
23 10 -85.59650846519047
23 20 -55.80121607782987
23 30 -35.128441712381175
23 40 -19.732747827585374


In [8]:
#最终结果由6个分类器投票表决，如果出现平局可以考虑每个分类器的置信度
def vote(x):
    votes = np.zeros(4)
    for param in params:
        p = predict(param, x)
        result = param[0]
        if p > 0.5:
            result = param[1]
        result = int(result)
        votes[result] += 1
    return votes.argmax()


vote(x[1]), y[1]

(1, 1)

In [9]:
#测试
correct = 0
for i in range(N):
    p = vote(x[i])
    if p == y[i]:
        correct += 1

print(correct / N)

0.99625
